In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np
from sklearn import svm
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint

d:\Anaconda3\envs\MLOntology\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
d:\Anaconda3\envs\MLOntology\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#global variabls

directory_path = "D:/MLOntology/NCIt/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"


In [3]:
import re
def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return m.group() if m else None

In [4]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptID = get_trailing_number(splitted[1])
                conceptLabelDict[conceptID] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_owl_ncit.txt"
read_label(label_file)
print(conceptLabelDict["4863"])
print(conceptLabelDict["115117"])
print(errors)

prostate carcinoma
stage ia esophageal cancer ajcc v7
[]


In [5]:
conceptPairDict={}
errors=[]
conceptPairList=[]

def read_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                childID = get_trailing_number(splitted[1])
                parentID = get_trailing_number(splitted[2].replace("\r\n", ""))
                conceptPairList.append([childID, parentID , 1])
#                 conceptPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

pair_file = data_path + "ontHierarchy_owl_ncit.txt"
read_pair(pair_file)

checkpairs = conceptPairList[10:15]
print(checkpairs)
print(len(conceptPairList))

[['4861', '7318', 1], ['87152', '87150', 1], ['87153', '140032', 1], ['87154', '87153', 1], ['87155', '87153', 1]]
16533


In [6]:
conceptNotPairDict={}
conceptNotPairList=[]

def read_not_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==2:
                childID = get_trailing_number(splitted[0])
                notparentID = get_trailing_number(splitted[1].replace("\r\n", ""))
                conceptNotPairList.append([childID, notparentID, 0])
#                 conceptNotPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

notPair_file = data_path + "taxNotPairs_owl_ncit.txt"
read_not_pair(notPair_file)


first2pairs =conceptNotPairList[10:15]
print(first2pairs)
print(len(conceptNotPairList))

[['86053', '88414', 0], ['8784', '7917', 0], ['141358', '141347', 0], ['141358', '141353', 0], ['141358', '141350', 0]]
9533


In [7]:
# In-place shuffle
random.shuffle(conceptNotPairList)
conceptNotPairList = conceptNotPairList[:len(conceptPairList)]

print(len(conceptNotPairList))

9533


In [8]:
vector_model_file = vector_model_path + "model0"

vector_model = gensim.models.Doc2Vec.load(vector_model_file)

inferred_vector = vector_model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(vector_model.docvecs.most_similar([inferred_vector], topn=10))

[('142823', 0.7903113961219788),
 ('3497', 0.7410315275192261),
 ('4566', 0.7290605902671814),
 ('5671', 0.7240594625473022),
 ('27710', 0.7174608707427979),
 ('5662', 0.7053307294845581),
 ('4888', 0.7046777009963989),
 ('4453', 0.7041112184524536),
 ('7527', 0.7026574611663818),
 ('8531', 0.6967900991439819)]


In [9]:
vector_model.docvecs[7918]

array([-0.00466427, -0.2133093 , -0.13188908,  0.0122558 ,  0.06393942,
       -0.13639419,  0.17377363,  0.20860074,  0.05809897,  0.04649761,
       -0.16894959,  0.04225196,  0.02165174,  0.00213318,  0.23631908,
        0.16286871, -0.05403031,  0.17041123, -0.05465656, -0.13113794,
       -0.17594796, -0.01543587, -0.06278177,  0.06830045, -0.09180123,
        0.13543412, -0.08404744, -0.05415916,  0.01433831,  0.18743008,
        0.0728869 , -0.03073151,  0.01052943, -0.06885084, -0.07429604,
       -0.04760221, -0.05925706, -0.02305547, -0.00136033,  0.09988423,
        0.18765847,  0.07289595, -0.18184835,  0.06907446, -0.02444701,
       -0.10360537,  0.06086358,  0.2055683 ,  0.02755826, -0.23605673,
        0.03812493, -0.03434969,  0.05227038, -0.07879271, -0.08301589,
       -0.02411955,  0.05062204,  0.14589028,  0.08253404, -0.09271763,
        0.10263631, -0.02226311, -0.08953032, -0.16695775,  0.24429363,
       -0.21860647, -0.0721235 , -0.22428198, -0.11841042,  0.05

In [10]:
vector_model_file = vector_model_path + "model1"

vector_model = gensim.models.Doc2Vec.load(vector_model_file)

inferred_vector = vector_model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
pprint(vector_model.docvecs.most_similar([inferred_vector], topn=10))

[('3725', 0.6395902037620544),
 ('45514', 0.6395784616470337),
 ('139545', 0.5937005281448364),
 ('5665', 0.5892391800880432),
 ('45652', 0.5882729291915894),
 ('5658', 0.5669200420379639),
 ('3995', 0.5668214559555054),
 ('45631', 0.5628659725189209),
 ('90513', 0.5613369941711426),
 ('36305', 0.5602449774742126)]


In [11]:
feature_number = 1024

def readFromPairList(id_pair_list, id_notPair_list):
    pair_list = id_pair_list + id_notPair_list
    random.shuffle(pair_list)
    idpairs_list =[]
    label_list =[]
    for i, line in enumerate(pair_list):      
        idpairs_list.append([line[0], line[1]])
        label_list.append(line[2])
    return idpairs_list, label_list

idpairs_list, label_list= readFromPairList(conceptPairList, conceptNotPairList)

print(label_list[:20])

[0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(idpairs_list, label_list, test_size = 0.2, shuffle= True)
print(X_train[:20])
print(X_train[:20])
print(y_train[:20])
print(y_test[:20])

[['5055', '7611'], ['8921', '27786'], ['68666', '27268'], ['5787', '135090'], ['38150', '7148'], ['139744', '139753'], ['5730', '8210'], ['7080', '7378'], ['8060', '7714'], ['6081', '4653'], ['3469', '6814'], ['9144', '68689'], ['8726', '8106'], ['96878', '7130'], ['5628', '5629'], ['3403', '121619'], ['5820', '6421'], ['7244', '7237'], ['3856', '9011'], ['9167', '3692']]
[['5055', '7611'], ['8921', '27786'], ['68666', '27268'], ['5787', '135090'], ['38150', '7148'], ['139744', '139753'], ['5730', '8210'], ['7080', '7378'], ['8060', '7714'], ['6081', '4653'], ['3469', '6814'], ['9144', '68689'], ['8726', '8106'], ['96878', '7130'], ['5628', '5629'], ['3403', '121619'], ['5820', '6421'], ['7244', '7237'], ['3856', '9011'], ['9167', '3692']]
[1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1]


In [25]:
feature_number = 1024

def getVectorFromModel(concept_id, conceptLabelDict, model):
    if concept_id in model.docvecs:
        concept_vector= model.docvecs[concept_id]
    else:
        print("%s not found, get inferred vector "%(concept_id))
        concept_label = conceptLabelDict[concept_id]
        concept_vector= model.infer_vector(concept_label.split())
    return concept_vector

def getVector(line, conceptLabelDict, model):        
    a = getVectorFromModel(line[0], conceptLabelDict, model)
    b = getVectorFromModel(line[1], conceptLabelDict, model)
    c = np.array((a, b))
    c = c.T 
#     c = np.expand_dims(c, axis=2)
    print(c.shape)
    return c

In [14]:
def stackVector(vector):
    from numpy import dstack
    return dstack((vector, vector, vector))

In [26]:
nb_classes=2 

def generator(x_samples, y_samples, train_flag, batch_size=64):
    samples = list(zip(x_samples, y_samples))
    num_samples = len(samples)
    
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            X_samples = []
            Y_samples= []
            for batch_sample in batch_samples:
                pair_list = batch_sample[0]
                data_vector = getVector(pair_list, conceptLabelDict, vector_model)
                data_vector = stackVector(data_vector)
                print(data_vector.shape)
                X_samples.append(data_vector)
                class_label = batch_sample[1] 
                Y_samples.append(class_label)
                
            X_samples = np.array(X_samples).astype('float32')
            Y_samples = np.eye(nb_classes)[Y_samples]
            print('one batch ready')
            yield shuffle(X_samples, Y_samples)

In [27]:
set_batch_size = 64

# compile and train the model using the generator function
train_generator = generator(X_train, y_train, train_flag=True, batch_size=set_batch_size)
validation_generator = generator(X_test, y_test, train_flag=False, batch_size=set_batch_size)

In [17]:
from keras import applications
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

import numpy as np

In [ ]:
# build the model??




In [18]:
model = applications.resnet50.ResNet50(weights=None, include_top = True, classes=2)
print('Model loaded')

for layer in model.layers:
    layer.trainable = True
    
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

Model loaded


In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger(directory_path + 'resnet18_cifar10.csv')

In [28]:
myepochs = 10

model.fit_generator(train_generator, 
                    steps_per_epoch=len(X_train)//set_batch_size, 
                    epochs=myepochs,
                    validation_data=validation_generator,
                    validation_steps=len(X_test)//set_batch_size,
                    callbacks=[lr_reducer, early_stopper, csv_logger])

Epoch 1/10
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(200, 2)
(200, 2, 3)
(2

ValueError: Error when checking input: expected input_1 to have shape (224, 224, 3) but got array with shape (200, 2, 3)

In [ ]:
import numpy
# vstack
from numpy import vstack
from numpy import hstack

a1 = np.zeros((1, 2,512))
# print(a1)
print(a1.shape)
a2 = np.ones((1, 2,512))
# print(a2)
print(a2.shape)
a3 = np.ones((1, 2,512))
# print(a3)
print(a3.shape)
a4 = vstack((a1, a2, a3))
print(a4)
print(a4.shape)


a1 = np.zeros((512,1))
# print(a1)
print(a1.shape)
a2 = np.ones((512,1))
# print(a2)
print(a2.shape)
a3 = np.ones((512,1))
# print(a3)
print(a3.shape)
a4 = hstack((a1, a2, a3))
print(a4)
print(a4.shape)

In [ ]:
import numpy
# vstack
from numpy import vstack
from numpy import hstack
from numpy import dstack

a1 = np.zeros((2, 1))
# print(a1)
print(a1.shape)
a2 = np.ones((2, 1))
# print(a2)
print(a2.shape)

a3 = hstack((a1, a2))
# print(a3)
print(a3.shape)

a4 = dstack((a3, a3, a3))
print(a4.shape)
print(a4)
